<a href="https://colab.research.google.com/github/vimalkumarasamy/computer_vision/blob/master/CV_Task2_OCR_single_threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/Computer_Vision

/content/gdrive/My Drive/Computer_Vision


In [3]:
ls

a.jpg      b_new.jpg   c_new.jpg        proj1-task2-png.png  utils.py
a_new.jpg  b.pgm       c_.pgm           __pycache__/
a.pgm      c_grey.jpg  c.pgm            task1.py
b.jpg      c.jpg       proj1-task1.jpg  task2.py


In [0]:
import argparse
import copy
import os
import cv2
import numpy as np
import utils
from task1 import *
from matplotlib import pyplot as plt

def arg_max(arr,c):
  m=[]
  (l,b)=arr.shape
  for i in range(l):
    for j in range(b):
      if arr[i][j]>=c:
        m+=[(i,j)]
  return(m)


In [0]:
matches=[]
img_path='proj1-task2-png.png'
gaussian=np.matmul(np.array([[1],[2],[1]]),np.array([[1,2,1]]))
img=read_image(img_path)
img_noise_reduced=convolve2d(img,gaussian)

template = cv2.imread("a.jpg",0)
resized = normalize(cv2.resize(template, (11,11), interpolation=cv2.INTER_LINEAR).tolist())
resized_threshold=copy.deepcopy(resized)
for i in range(len(resized)):
  for j in range(len(resized[0])):
    if resized[i][j]<100:
      resized_threshold[i][j]=0
    else:
      resized_threshold[i][j]=255
img_ncc=ncc(img_noise_reduced,resized_threshold)
img_ncc_np=np.array(img_ncc)
matches+=arg_max(img_ncc_np,0.57)

resized = normalize(cv2.resize(template, (18,18), interpolation=cv2.INTER_LINEAR).tolist())
resized_threshold=copy.deepcopy(resized)
for i in range(len(resized)):
  for j in range(len(resized[0])):
    if resized[i][j]<100:
      resized_threshold[i][j]=0
    else:
      resized_threshold[i][j]=255
img_ncc=ncc(img_noise_reduced,resized_threshold)
img_ncc_np=np.array(img_ncc)
matches+=arg_max(img_ncc_np,0.65)


In [0]:
# Lets check for template b
matches=[]
img_path='proj1-task2-png.png'
gaussian=np.matmul(np.array([[1],[2],[1]]),np.array([[1,2,1]]))
img=read_image(img_path)
img_noise_reduced=convolve2d(img,gaussian)

template = cv2.imread("b_new.jpg",0)
# resized = normalize(cv2.resize(template, (14,9), interpolation=cv2.INTER_LINEAR).tolist())
resized=template
resized_threshold=copy.deepcopy(resized)
for i in range(len(resized)):
  for j in range(len(resized[0])):
    if resized[i][j]<130:
      resized_threshold[i][j]=0
    else:
      resized_threshold[i][j]=255
img_ncc=ncc(img_noise_reduced,resized_threshold)
img_ncc_np=np.array(img_ncc)
# matches=matches+np.argwhere(img_ncc_np > 0.7).tolist()
matches+=arg_max(img_ncc_np,0.7)


[(147, 36), (192, 55)]

In [0]:
# Lets check for template c
matches=[]
img_path='proj1-task2-png.png'
gaussian=[[1,2,1],[2,4,2],[1,2,1]]
img=read_image(img_path)
img_noise_reduced=convolve2d(img,gaussian)

# template = cv2.imread("c_.pgm",0)
template = cv2.imread("c.jpg",0)
resized = normalize(cv2.resize(template, (10,10), interpolation=cv2.INTER_LINEAR).tolist())
# resized=template
resized_threshold=copy.deepcopy(resized)
for i in range(len(resized)):
  for j in range(len(resized[0])):
    if resized[i][j]<100:
      resized_threshold[i][j]=0
    else:
      resized_threshold[i][j]=255
img_ncc=ncc(img_noise_reduced,resized_threshold)
img_ncc_np=np.array(img_ncc)
matches+=arg_max(img_ncc_np,0.52)
print(matches)

[(14, 87), (14, 162), (14, 219), (14, 220), (59, 55), (59, 263), (178, 164), (200, 93), (200, 159), (200, 214), (201, 93), (245, 37), (245, 50), (245, 103), (245, 216), (246, 165), (246, 179), (268, 42), (268, 110), (269, 86)]


In [0]:
# Look at the size of the template and identify the alphabet => customize the cutoff based on that => Poka Poka!

In [0]:
def detect(img,template):
  import numpy as np
  import utils
  
  def correlation_coefficient(patch1, patch2):
      product = np.mean((patch1 - patch1.mean()) * (patch2 - patch2.mean()))
      stds = patch1.std() * patch2.std()
      if stds == 0:
          return 0
      else:
          product /= stds
          return product
    
  def ncc(image,temp):
    import numpy as np
    t_np=np.array(temp)
    i_np=np.array(image)
    t_np=t_np/t_np.sum()
    i_l=len(image)
    i_b=len(image[0])
    t_l=len(temp)
    t_b=len(temp[0])
    ncc_mat=np.zeros((i_l-t_l+1,i_b-t_b+1)).tolist()
    for i in range(i_l-t_l+1):
      for j in range(i_b-t_b+1):
        ncc_mat[i][j]=correlation_coefficient(i_np[i:i+t_l,j:j+t_b],t_np)
    return(ncc_mat)

  def arg_max(arr,c):
    m=[]
    (l,b)=arr.shape
    for i in range(l):
      for j in range(b):
        if arr[i][j]>=c:
          m+=[(i,j)]
    return(m)

  def remove_duplicates(m):
    dedup=[]
    for i in m:
      p,q=i[0],i[1]
      if (p-1,q) in dedup or (p+1,q) in dedup or (p,q-1) in dedup or (p,q+1) in dedup or (p+1,q+1) in dedup or (p-1,q-1) in dedup or (p+1,q-1) in dedup or (p-1,q+1) in dedup:
        pass 
      else: 
        dedup+=([i])
    return(dedup)


  (x,y)=np.array(template).shape
  gaussian=np.array([[1,2,1],[2,4,2],[1,2,1]])
  if (x,y)==(62, 55):
    matches=[]
    template=np.array(template).astype('float32')
    img_noise_reduced=convolve2d(img,gaussian)
    resized = normalize(cv2.resize(template, (11,11), interpolation=cv2.INTER_LINEAR).tolist())
    resized_threshold=copy.deepcopy(resized)
    for i in range(len(resized)):
      for j in range(len(resized[0])):
        if resized[i][j]<100:
          resized_threshold[i][j]=0
        else:
          resized_threshold[i][j]=255
    img_ncc=ncc(img_noise_reduced,resized_threshold)
    img_ncc_np=np.array(img_ncc)
    matches+=arg_max(img_ncc_np,0.57)

    resized = normalize(cv2.resize(template, (18,18), interpolation=cv2.INTER_LINEAR).tolist())
    resized_threshold=copy.deepcopy(resized)
    for i in range(len(resized)):
      for j in range(len(resized[0])):
        if resized[i][j]<100:
          resized_threshold[i][j]=0
        else:
          resized_threshold[i][j]=255
    img_ncc=ncc(img_noise_reduced,resized_threshold)
    img_ncc_np=np.array(img_ncc)
    matches+=arg_max(img_ncc_np,0.65)
    return(remove_duplicates(matches))

  if (x,y)==(20, 16):
    matches=[]
    img_noise_reduced=convolve2d(img,gaussian)
    resized=template
    resized_threshold=copy.deepcopy(resized)
    for i in range(len(resized)):
      for j in range(len(resized[0])):
        if resized[i][j]<130:
          resized_threshold[i][j]=0
        else:
          resized_threshold[i][j]=255
    img_ncc=ncc(img_noise_reduced,resized_threshold)
    img_ncc_np=np.array(img_ncc)
    matches+=arg_max(img_ncc_np,0.7)
    return(remove_duplicates(matches))
  
  if (x,y)==(54,37):
    matches=[]
    template=np.array(template).astype('float32')
    img_noise_reduced=convolve2d(img,gaussian)
    resized = normalize(cv2.resize(template, (10,10), interpolation=cv2.INTER_LINEAR).tolist())
    resized_threshold=copy.deepcopy(resized)
    for i in range(len(resized)):
      for j in range(len(resized[0])):
        if resized[i][j]<100:
          resized_threshold[i][j]=0
        else:
          resized_threshold[i][j]=255
    img_ncc=ncc(img_noise_reduced,resized_threshold)
    img_ncc_np=np.array(img_ncc)
    matches+=arg_max(img_ncc_np,0.52)
    return(remove_duplicates(matches))



In [0]:
img=cv2.imread('proj1-task2-png.png',0)
template=cv2.imread('a.jpg',0)
detect(img.tolist(),template.tolist())

[(13, 152),
 (13, 169),
 (36, 91),
 (58, 121),
 (58, 227),
 (81, 88),
 (81, 150),
 (132, 170),
 (132, 215),
 (154, 122),
 (154, 164),
 (177, 79),
 (177, 155),
 (199, 84),
 (199, 150),
 (199, 167)]